<a href="https://colab.research.google.com/github/keshav20004/NLP-LSTM/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohamadalhasan/a-fake-news-dataset-around-the-syrian-war")

print("Path to dataset files:", path)

100%|██████████| 458k/458k [00:00<00:00, 616kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/mohamadalhasan/a-fake-news-dataset-around-the-syrian-war/versions/1


In [ ]:
import pandas as pd



In [ ]:
df=pd.read_csv("//content/FA-KES-Dataset.csv", encoding='latin1')

In [ ]:
df.head()

,unit_id,article_title,article_content,source,date,location,labels
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,4/5/2017,idlib,0
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,nna,4/7/2017,homs,0
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,nna,4/16/2017,aleppo,0
3,1914947534,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,nna,4/19/2017,aleppo,0
4,1914947535,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,nna,7/10/2016,aleppo,0


In [ ]:
df.shape

(804, 7)

In [ ]:
df.isnull().sum()

,0
unit_id,0
article_title,0
article_content,0
source,0
date,0
location,0
labels,0


In [ ]:
## Get the Independent Features

X=df.drop('labels',axis=1)

In [ ]:
## Get the Dependent features
y=df['labels']

In [ ]:
X.shape

(804, 6)

In [ ]:
y.shape

(804,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.19.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [ ]:
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages["article_title"][1]

'Homs governor says U.S. attack caused deaths but doesnt see big human losses'

In [ ]:
messages

,unit_id,article_title,article_content,source,date,location
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,4/5/2017,idlib
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,nna,4/7/2017,homs
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,nna,4/16/2017,aleppo
3,1914947534,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,nna,4/19/2017,aleppo
4,1914947535,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,nna,7/10/2016,aleppo
...,...,...,...,...,...,...
799,1965511221,Turkish Bombardment Kills 20 Civilians in Syria,28-08-2016 Turkish Bombardment Kills 20 Civili...,manar,8/28/2016,aleppo
800,1965511222,Martyrs as Terrorists Shell Aleppos Salah Eddin,17-08-2016 Martyrs as Terrorists Shell Aleppos...,manar,8/1/2016,aleppo
801,1965511224,Chemical Attack Kills Five Syrians in Aleppo SANA,03-08-2016 Chemical Attack Kills Five Syrians ...,manar,8/3/2016,aleppo
802,1965511226,5 Killed as Russian Military Chopper Shot down...,01-08-2016 5 Killed as Russian Military Choppe...,manar,8/1/2016,idlib


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
##data preprocessing i.e stopwords remove krna
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
corpus=[]
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]'," ",messages["article_title"][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word)for word in review if not word in stopwords.words("english")]
  review=' '.join(review)
  corpus.append(review)

In [ ]:
corpus[1]

'om governor ys . . tt ck c use de th doesnt see big hum n loss'

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
one_hot

<function keras.src.legacy.preprocessing.text.one_hot(input_text, n, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', analyzer=None)>

### Replacing Deprecated `one_hot` with Modern `Tokenizer` and `pad_sequences`

The `keras.preprocessing.text.one_hot` function is deprecated. For robust text preprocessing in TensorFlow/Keras, it's recommended to use `tf.keras.preprocessing.text.Tokenizer` for converting text to sequences of integers, and `tf.keras.preprocessing.sequence.pad_sequences` to ensure all input sequences have a uniform length, which is essential for neural network architectures like LSTMs.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer
# `num_words` is set to `voc_size` (5000) to keep the vocabulary size consistent.
# `oov_token` (out-of-vocabulary token) handles words not in the vocabulary.
tokenizer = Tokenizer(num_words=voc_size, oov_token="<unk>")

# Fit the tokenizer on the preprocessed corpus
tokenizer.fit_on_texts(corpus)

# Convert texts in the corpus to sequences of integers
tokenized_sequences = tokenizer.texts_to_sequences(corpus)

# Determine the maximum sequence length in the tokenized corpus.
# This length will be used for padding all sequences to a consistent size.
max_length = max(len(seq) for seq in tokenized_sequences)
print(f"Maximum sequence length after tokenization: {max_length}")

# Pad the sequences to ensure uniform length.
# 'padding="post"' means zeros are added at the end of shorter sequences.
# 'maxlen' sets the length of all sequences to `max_length`.
padded_sequences = pad_sequences(tokenized_sequences, maxlen=max_length, padding='post')

print("\nExample of a tokenized and padded sequence (first entry):")
print(padded_sequences[0])
print("Shape of padded_sequences:", padded_sequences.shape)

# Now, `padded_sequences` is ready to be used as input for the Embedding layer in your model.

Maximum sequence length after tokenization: 20

Example of a tokenized and padded sequence (first entry):
[  2   8   9 542 543 394 544 203   0   0   0   0   0   0   0   0   0   0
   0   0]
Shape of padded_sequences: (804, 20)


The `padded_sequences` variable now holds the preprocessed numerical representation of your text data, ready for an embedding layer. You should use `padded_sequences` instead of `onehot_repr` in your next steps.

In [ ]:
onehot_repr[1]

[774,
 4462,
 1236,
 1581,
 2563,
 1517,
 3339,
 2170,
 4997,
 889,
 4301,
 3595,
 937,
 3759,
 3281]

In [ ]:
#Embedding

In [ ]:
sent_length=20
embeddeb_docs=pad_sequences(onehot_repr,padding="pre",maxlen=sent_length)
print(embeddeb_docs)

[[   0    0    0 ... 1422 4879 3339]
 [   0    0    0 ...  937 3759 3281]
 [   0    0    0 ... 3712  751 1365]
 ...
 [   0    0    0 ... 2152 2848  254]
 [   0    0    0 ... 3453  967 2152]
 [   0    0    0 ... 3369 3208 2670]]


In [ ]:
embeddeb_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 2152, 1581, 2563, 3629,  755, 1422, 4879, 3339], dtype=int32)

In [ ]:
##Creating a model

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
len(embeddeb_docs),y.shape

(804, (804,))

In [ ]:
import numpy as np
X_final=np.array(embeddeb_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((804, 20), (804,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 77ms/step - accuracy: 0.5134 - loss: 0.6934 - val_accuracy: 0.5639 - val_loss: 0.6914
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5665 - loss: 0.6904 - val_accuracy: 0.5564 - val_loss: 0.6889
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5230 - loss: 0.6886 - val_accuracy: 0.5564 - val_loss: 0.6890
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5474 - loss: 0.6839 - val_accuracy: 0.5602 - val_loss: 0.6883
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5951 - loss: 0.6694 - val_accuracy: 0.4624 - val_loss: 0.6977
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6687 - loss: 0.6424 - val_accuracy: 0.4549 - val_loss: 0.7422
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5966 - loss: 0.6493 - val_accuracy: 0.5677 - val_loss: 0.6949
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6723 - loss: 0.6259 - val_accuracy: 0.5451 - val_loss: 0.7034


In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
y_pred=model.predict(X_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [ ]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[118,   0],
       [148,   0]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.44360902255639095

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.44      1.00      0.61       118
           1       0.00      0.00      0.00       148

    accuracy                           0.44       266
   macro avg       0.22      0.50      0.31       266
weighted avg       0.20      0.44      0.27       266



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
